# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-3.5-Turbo-1106` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-3.5 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-3.5-turbo-1106&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [7]:
from openai import OpenAI
client = OpenAI()

In [2]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

## 使用 LangChain 构造训练数据

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [2]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [3]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [4]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [11]:
ai_message = chat(messages)

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


### 解析 OpenAI GPT 生成的训练数据

In [12]:
ai_message.content

'content:"蒙卦"\nsummary:"在周易中，蒙卦代表着教育的启蒙智慧。它由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。艮为山的形象，意味着停止；坎为水的形象，意味着险难。卦形为山下有险，却仍然不停止前进，象征着蒙昧的状态，因此得名蒙卦。然而，通过把握时机，行动与时俱进，蒙卦具有启蒙和通达的卦象。\n\n蒙卦在《序卦》中位于始生卦《屯》之后，解释为“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”意味着万物在幼稚阶段都处于蒙昧未开的状态，而人类则特指儿童的启蒙阶段。\n\n《象》中解释蒙卦为“山下出泉，蒙；君子以果行育德”，意味着像泉水从山下涌出一样，启蒙过程就像是培育品德的过程，需要果断的行动。"\n'

In [13]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 在周易中，蒙卦代表着教育的启蒙智慧。它由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。艮为山的形象，意味着停止；坎为水的形象，意味着险难。卦形为山下有险，却仍然不停止前进，象征着蒙昧的状态，因此得名蒙卦。然而，通过把握时机，行动与时俱进，蒙卦具有启蒙和通达的卦象。

蒙卦在《序卦》中位于始生卦《屯》之后，解释为“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”意味着万物在幼稚阶段都处于蒙昧未开的状态，而人类则特指儿童的启蒙阶段。

《象》中解释蒙卦为“山下出泉，蒙；君子以果行育德”，意味着像泉水从山下涌出一样，启蒙过程就像是培育品德的过程，需要果断的行动。


### 持久化存储训练数据集文件

In [14]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [15]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [17]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [18]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [21]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-3.5调用
chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [19]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content:"蒙卦"
summary:"在周易中，蒙卦代表着通泰之象。它是由艮卦（山）上卦和坎卦（水）下卦相叠而成。蒙卦的核心意义在于强调，不是我求告于幼稚愚昧的人，而是幼稚愚昧的人向我求告。第一次占卜会得到神灵的启示，但如果轻慢不敬地再三占卜，那神灵就不会给予回应。然而，这个卦象仍然是吉利的。

        蒙卦的象征意义是山下有泉，泉水喷涌而出。君子观此卦象，以果行育德，取法于一往无前的山泉，通过果敢坚毅的行动来培养自身的品德。

        《断易天机》解释：蒙卦为艮卦上坎卦下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。

        进一步，北宋易学家邵雍解释：蒙卦象征着智慧未开，蒙昧闭塞，犹豫不决，缺乏果断。得到此卦的人，智慧犹如童蒙，不辨是非，迷失方向，然而如果能顺应贤师良友之教，启其聪明则会通达。

        台湾国学大儒傅佩荣提到，时运是蓄积德行，出而用世；财运是矿山生意，果决则吉；家宅如君子居吉，婚姻之始；身体方面，要驱除邪热，可保平安。

        蒙卦在传统解卦中描述为山下有险而不停止前进，强调抓住时机，切合时宜来培养通达和启蒙的卦象。运势初时迷惑不知方向，需要忍耐待机而动，多听取别人意见，才能通达。在事业、经商、求名、婚恋等方面，皆需果敢坚毅、谨慎等品质，方能有顺利发展。"


In [23]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [20]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 蒙卦
Summary: 在周易中，蒙卦代表着通泰之象。它是由艮卦（山）上卦和坎卦（水）下卦相叠而成。蒙卦的核心意义在于强调，不是我求告于幼稚愚昧的人，而是幼稚愚昧的人向我求告。第一次占卜会得到神灵的启示，但如果轻慢不敬地再三占卜，那神灵就不会给予回应。然而，这个卦象仍然是吉利的。

        蒙卦的象征意义是山下有泉，泉水喷涌而出。君子观此卦象，以果行育德，取法于一往无前的山泉，通过果敢坚毅的行动来培养自身的品德。

        《断易天机》解释：蒙卦为艮卦上坎卦下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。

        进一步，北宋易学家邵雍解释：蒙卦象征着智慧未开，蒙昧闭塞，犹豫不决，缺乏果断。得到此卦的人，智慧犹如童蒙，不辨是非，迷失方向，然而如果能顺应贤师良友之教，启其聪明则会通达。

        台湾国学大儒傅佩荣提到，时运是蓄积德行，出而用世；财运是矿山生意，果决则吉；家宅如君子居吉，婚姻之始；身体方面，要驱除邪热，可保平安。

        蒙卦在传统解卦中描述为山下有险而不停止前进，强调抓住时机，切合时宜来培养通达和启蒙的卦象。运势初时迷惑不知方向，需要忍耐待机而动，多听取别人意见，才能通达。在事业、经商、求名、婚恋等方面，皆需果敢坚毅、谨慎等品质，方能有顺利发展。


In [24]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [25]:
# 执行主函数
main()

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 蒙卦
Summary: 在周易中，蒙卦代表通泰，卜问吉利。这是一个由艮卦（山）上卦和坎卦（水）下卦组成的异卦相叠。卦辞中描述了对幼稚愚昧的人有求于我的情形，同时强调了神灵的告知需要一次就足够，如果轻慢不敬地再三占筮，神灵就不会告诉他。蒙卦的象辞中，艮卦象征着山，坎卦象征着泉水，山下有泉喷涌而出，寓意着蒙卦代表了一种果敢行动培养品德的精神。

蒙卦象征着初时迷惑不知方向，需要耐心待机而动。吉凶解中表示，凡事多听取别人意见，则运可通。在事业上，必须勇敢坚毅，接受严格教育，务必脚踏实地。在经商过程中应小心谨慎，树立良好商业道德，以信誉取胜。在求名、婚恋、决策方面也都强调了谨慎，接受教育和等待适当时机的重要性。"

comment:"非常全面的解读，包含了蒙卦的内涵、象辞、历代注解和各个方面的运势解析，可以作为大模型的训练数据。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 屯卦
Summary: 在周易中，屯卦的卦象是由坎卦与震卦组成的异卦，坎为云，震为雷。这是一个预示困难和险境的卦象，暗示万事始生时所面临的艰难险阻，然而顺应时机则必能蓬勃发展。屯卦代表了困难之始，需要坚忍不拔的毅力和果敢的行动才能带来吉利。

屯卦的核心哲学是：面临困境时，需要步步为营，勇往直前，灵活机动，方可获得成功。这是一个初难后解的卦象，表明虽然起初困难重重，但随着时机的到来，可以摆脱困境，获得成功。值得注意的是，在困难时期应多施恩惠，同时需要得到他人的协助。

屯卦的卦象也与事业、经商、求名、婚姻等方面有关，主要意味着初始困难，需要坚定信念、积极进取，以及等待时机的智慧。肯定的积极性和不屈不挠的毅力将会带来美好的结果。"

reference:[
"《象辞》：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。",
"《断易天机》：屯卦坎上震下，为坎宫二世卦。屯说明有困难，又象征动而逢险境，需刚毅果敢方为吉。",
"北宋易学家邵雍解：万物始生，开始困难；先劳后逸，苦尽甘来。得此卦者，身处困境，宜守不宜进，须多加辛苦努力，排除困难，方可通达，有初难后解之象。",
"台湾国学大儒傅佩荣解：时运宜守不宜进，财运创业维艰，家宅修缮住宅；初婚不和，身体保存元气。",
"传统解卦：这个卦是异卦（下震上坎）相叠，震为雷，喻动；坎为雨，喻险。雷雨交加，险象丛生，环境恶劣。屯原指植物萌生大地，万物始生，充满艰难险阻，然而顺时应运，必欣欣向荣。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

Content: {
  "content": "比卦",
  "summary": "在周易中，比卦是一个代表着吉利的卦象。它由下卦坤（地）和上卦坎（水）组成。比卦暗示着再次进行卜筮，依然能够得到吉祥的结果，并预示长期坚定的吉利。同时，也提醒谨防不愿臣服的邦国迟迟不来朝，可能会带来麻烦。比卦的核心哲学是：相亲相依，长期亲密，就能够无灾无咎，所以预示吉祥。先王观此卦象，取法于水附大地，地纳江河之象，封建万国，亲近诸侯。",
  "interpretation": "比卦坎上坤下，为坤宫归魂卦。比为相亲相依附之意，长期如此，就会无咎，所以吉祥。比卦代表着平顺，身处一阳之下统管五阴，相亲相辅，和乐之象。运势方面，预示着平顺，能够得到贵人的提拔，建议在面对事务时迅速决策，不宜过于迟疑。在事业上，比卦暗示着顺利发展，可以获得他人的帮助和辅佐，需以诚实、信任的态度对待他人，向才德高尚的人士学习，听取建议。经商方面，愿望可以实现且利润丰厚，但需要与他人密切合作，真诚交往并遵守商业道德。在求名方面，有着成功的希望，更重要的是得到他人的赏识和栽培。婚姻方面，比卦预示着美好婚缘和彼此忠诚。总体而言，比卦提醒人们要心地善良，待人忠诚、厚道，乐于帮助他人，工作勤恳并善于判断是非，在交友选择上要谨慎，结交高明的朋友，会终身受益。",
  "image": "比"
}
``
Summary: {
  "content": "比卦",
  "summary": "在周易中，比卦是一个代表着吉利的卦象。它由下卦坤（地）和上卦坎（水）组成。比卦暗示着再次进行卜筮，依然能够得到吉祥的结果，并预示长期坚定的吉利。同时，也提醒谨防不愿臣服的邦国迟迟不来朝，可能会带来麻烦。比卦的核心哲学是：相亲相依，长期亲密，就能够无灾无咎，所以预示吉祥。先王观此卦象，取法于水附大地，地纳江河之象，封建万国，亲近诸侯。",
  "interpretation": "比卦坎上坤下，为坤宫归魂卦。比为相亲相依附之意，长期如此，就会无咎，所以吉祥。比卦代表着平顺，身处一阳之下统管五阴，相亲相辅，和乐之象。运势方面，预示着平顺，能够得到贵人的提拔，建议在面对事务时迅速决策，不宜过于迟疑。在事业上，比卦暗示着顺利发展，可以获得他人的帮助和辅佐，需以诚实、信任的态度对待他人，向才德高尚的人士学习，听取建议。经商方面，愿望可以实现且利

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 坤卦
Summary: 坤卦，源自《周易》的卦象之一，被视为大吉大利的卦象。它由两个坤卦叠加而成，代表着地、顺、牵引、接纳。在这个卦象中，大地承载万物，取法于地，以柔顺厚德来承担重大的责任。

坤卦的核心哲学是：顺应天道，柔顺地德。君子观此卦象，以深厚的德行来承担重大的责任。预示着旅行者初时可能迷失方向，后来找到主人，吉利。在行事上，西南得朋，东北丧朋。此外，定居安贞也预示吉利。

《断易天机》解释说：坤卦为地气舒展之象，具有纯阴之性，预示着宜往西南行，可得朋友。在南宋时期，易学家邵雍认为得此卦者，宜顺从运势，以静制动，不宜独立谋事，顺从他人，一起合作，可成大事。

整体而言，坤卦暗示着诸事不宜急进，以静制动为宜。在事业、经商、求名、婚恋等方面都适宜顺从天道、柔顺和谦和的态度，方能取得吉祥的结果。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 乾卦
Summary: 在周易中，乾卦是六十四卦之首，代表天，由六个阳爻构成，象征着刚健、健全、兴盛和强健。《象辞》中说：天道刚健，运行不已，君子观此卦象，应效法天道，不断自强。这一卦象暗示着大通而至正，但必须行正道，方可永远亨通。

乾卦的核心哲学是：刚健旺盛，发育之功，完事顺利。得此卦者，宜把握机会，争取成果，但务须警惕过于刚直和骄傲。不仅如此，在事业、经商、求名、婚恋等方面，都需要坚持刚健、正直、公允的实质，修养德行，积累知识，坚定信念，自强不息，方能克服困难，取得成功。

这一卦象预示着大吉大利，事业如日中天，但同时也提醒着人们必须警惕盛极必衰的道理，提高警惕，小心谨慎，冷静处世。只有如此，才能真正充分发挥才智，保证事业成功。


### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 GPT-3.5-Turbo-1106 生成结果发现问题

In [26]:
def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [27]:
# 执行主函数
main()

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 蒙卦
Summary: 在周易中，蒙卦是一个充满着启蒙和通达意义的卦象。它由下卦坎（水）和上卦艮（山）组成，形象为山下有险。这象征着初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。蒙卦代表着混乱无序的情况下，需要勇敢坚毅的行动来扭转局面，同时也需要接受严格教育，培养奋发图强的精神，务必脚踏实地，杜绝好高骛远。在事业、经商、求名、婚恋等方面，蒙卦都提示着需要基础教育的重要性，同时也需要耐心等待适当时机，并接受长辈的教诲和严酷的考验。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 屯卦
Summary: 在周易中，屯卦代表着大吉大利，它由下卦震（雷）上卦坎（水）组成。这一卦象象征着最初的困难和阻碍，但需要坚韧不拔的品性和果敢的行动才能取得吉祥。屯卦告诫着困境中需要勇往直前，小心翼翼，才可期待后续的顺利解决。

根据《象辞》解释，屯卦的卦象是由上坎下震组成，坎象征云、震象征雷。这象征着云行于上，雷动于下，君子观此卦象，取法于云雷，以云的恩泽，雷的威严来治理国事。

屯卦的解释中强调：身处困境时需要步步为营，有初难后解之象。事业上，积极进取、行动果断、勇往直前，灵活机动，可获得大的成功；同时需要他人相助。对于经商、求名、婚恋等方面，亦应坚忍不拔、积极追求，方可获得成功。然而，需要意识到困难状态下的需要得到贤德之人的帮助才能摆脱，以乐观主义精神处世，才能取得成就。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

Content: 比卦
Summary: 在周易中，比卦是一个吉利的卦象。它代表着同时再次卜筮，仍然大吉大利，预示长期的吉祥而没有灾祸。然而，也暗示着不愿臣服的邦国迟迟不来者将会有难。

比卦的核心哲学是：相亲相依，长期的亲密和宽容将带来吉祥。它反映了相亲相辅、亲密无间、和乐相处的道理。坎上坤下，水附大地，地纳河海，相互依赖，亲密无间，象征着和乐相处的意境。

在运势、事业、经商、求名、婚恋等方面，比卦都预示着顺利成功，得到他人的帮助和辅佐。需要待人宽厚、正直、诚实、信任，在与他人的交往中讲究道德，遵守信义。同时，也需要警惕选择朋友，避免与品行不端的人结交，而应与高明之人友好往来，并获得其帮助。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 坤卦
Summary: 坤卦是一个大吉大利的卦象，代表着柔顺、顺应天道、承载万物、伸展无穷的特性。坤卦的核心哲学是先迷失而后得到正确方向，因此在旅行中先迷后得，财物方面宜往西南。总体而言，坤卦代表安定、顺利和安宁。在事业上，需要以静制动，不宜急进，重视内心修养，容忍负重，以诚信和大度为原则。在经商方面，应稳健经营，遇挫折要总结经验，谨慎行事。在婚恋方面，适宜以柔克刚，以温柔待人，可得美好姻缘。在决策上，需要具备忠厚、温和的性格，待人真诚，可得到他人的帮助，但也要小心提防小人的伤害。


/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Content: 乾卦
Summary: 乾卦是《易经》六十四卦之首，由六个阳爻组成，象征天，代表刚健、强健、大通的意义。在《象辞》中指出，天行健，君子应当以自强不息为行事准则。乾卦预示着吉祥如意，但需要行正道才能永远亨通。古代解卦家如邵雍和傅佩荣都认为得乾卦者宜自强不息，名利双收，但需谨慎过于刚直之嫌。在传统解卦中，乾卦预示着事业兴盛强健，但也提示了盛极必衰的道理，教诲人们谨慎小心，保持冷静谦和的态度，以保证事业成功。在经商、求名、婚恋等方面也提出了相应的建议，强调应修德养德，保持正直与公允，自强不息，才能克服困难，取得成功。
